## Parte 1 - Implementando um Banco de Dados Relacional com SQLAlchemy

In [15]:
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import Column
from sqlalchemy import select
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import Double
from sqlalchemy import ForeignKey

In [16]:
Base = declarative_base()

In [17]:
class Cliente(Base):
    __tablename__ = "cliente"
    id = Column(Integer, primary_key=True)
    nome = Column(String(50), nullable=False)
    cpf = Column(String(11), nullable=False, unique=True)
    endereco = Column(String(100), nullable=False)

    conta = relationship(
        "Conta", back_populates="cliente", cascade="all, delete-orphan"
    )

    def __repr__(self):
        return f"Cliente(id={self.id}, nome={self.nome}, cpf={self.cpf}, endereco={self.endereco})"


class Conta(Base):
    __tablename__ = "conta"
    id = Column(Integer, primary_key=True)
    tipo = Column(String, nullable=False)
    agencia = Column(String, nullable=False)
    numero = Column(Integer, unique=True, nullable=False)
    id_cliente = Column(Integer, ForeignKey("cliente.id"), nullable=False)
    saldo = Column(Double)

    cliente = relationship("Cliente", back_populates="conta")

    def __repr__(self):
        return f"Conta(id={self.id}, agencia={self.agencia}, numero={self.numero}, saldo={self.saldo})"

### Conexao com o DB

In [18]:
engine = create_engine("sqlite://")

### Criando tabelas

In [19]:
Base.metadata.create_all(engine)

### Inspecionado o DB

In [20]:
insp = inspect(engine)

### Retornando o nome das tabelas

In [21]:
print(insp.get_table_names())

['cliente', 'conta']


### Insercao dos dados

In [22]:
with Session(engine) as session:
    Marcos = Cliente(
        nome= "Marcos",
        cpf= "44027770312",
        endereco= "Rua qualquercoisa, 68",
        conta= [Conta(
            tipo= "conta_corrente",
            agencia= "547",
            numero= 1245976,
            saldo= 100.00
        )]
    )

    Maria = Cliente(
        nome= "Maria",
        cpf= "34017975328",
        endereco= "Lugar nenhum, 68",
        conta= [Conta(
            tipo= "conta_corrente",
            agencia= "789",
            numero= 1346798,
            saldo= 500.00
        )]
    )

    Julio = Cliente(
        nome= "Julio",
        cpf= "71517177388",
        endereco= "Lugar nenhum, 68",
        conta= [Conta(
            tipo= "conta_poupanca",
            agencia= "565",
            numero= 1597536,
            saldo= 1500.00
        )]
    )

### Enviando dados para o DB

In [23]:
session.add_all([Marcos, Maria, Julio])
session.commit()

### Recuperando dados das tabelas

In [25]:
dados = select(Cliente).where(Cliente.nome.in_(["Marcos", "Julio"]))
for dado in session.scalars(dados):
    print(dado)

Cliente(id=1, nome=Marcos, cpf=44027770312, endereco=Rua qualquercoisa, 68)
Cliente(id=3, nome=Julio, cpf=71517177388, endereco=Lugar nenhum, 68)


In [26]:
dados = select(Conta).where(Conta.id.in_([2]))
for dado in session.scalars(dados):
    print(dado)

Conta(id=2, agencia=789, numero=1346798, saldo=500.0)


### Manipulando Dados

In [27]:
dados_ordem = select(Cliente).order_by(Cliente.nome)
for ordem in session.scalars(dados_ordem):
    print(ordem)

Cliente(id=3, nome=Julio, cpf=71517177388, endereco=Lugar nenhum, 68)
Cliente(id=1, nome=Marcos, cpf=44027770312, endereco=Rua qualquercoisa, 68)
Cliente(id=2, nome=Maria, cpf=34017975328, endereco=Lugar nenhum, 68)


In [30]:
connection = engine.connect()

dados_join = select(Cliente.nome, Conta.tipo, Conta.numero, Conta.saldo).join_from(Conta, Cliente)
dados = connection.execute(dados_join).fetchall()
for dado in dados:
    print(dado)

('Marcos', 'conta_corrente', 1245976, 100.0)
('Maria', 'conta_corrente', 1346798, 500.0)
('Julio', 'conta_poupanca', 1597536, 1500.0)
